## Extract CERFA variables from Openfisca-France repo (and apply some manual tweaks)

In [ ]:
import pkgutil
import json
import re
import requests
from bs4 import BeautifulSoup

from openfisca_france import model
from openfisca_france.model.base import Variable

from deuxpots import CERFA_VARIABLES_PATH

Import all modules from OpenFisca-France model (so that __subclass__ finds them).

In [ ]:
for loader, module_name, is_pkg in pkgutil.walk_packages(model.__path__):
    if module_name not in globals():
        _module = loader.find_module(module_name).load_module(module_name)
        globals()[module_name] = _module

In [ ]:
variables = []
for cls in Variable.__subclasses__():
    try:
        cerfa = cls.cerfa_field
    except AttributeError:
        continue
    if isinstance(cerfa, dict):
        boxes = [cerfa[k] for k in sorted(cerfa)]
    elif isinstance(cerfa, str):
        boxes = [cerfa]
    else:
        raise ValueError("cerfa_field is neither a dict nor a string.")
    if len(boxes) == 1 and len(boxes[0]) == 1:  # Exclude single-letter "family" boxes (to be added manually later on)
        continue
    var_type = cls.value_type.__name__
    variable = {
        'boxes': tuple(sorted(boxes)),
        'type': 'int' if var_type == 'float' else var_type,
        'description': getattr(cls, 'label', None),
    }
    variables.append(variable)

Scrape variables from the HTML page of the tax simulator.

In [ ]:
response = requests.get('https://simulateur-ir-ifi.impots.gouv.fr/calcul_impot/2023/complet/index.htm')
soup = BeautifulSoup(response.text)

In [ ]:
REGEX_BOXCODE = re.compile(r'[0-9][A-Z]{2}')
boxcodes_scraped = [input['title'] for input in soup.select('input[type=text]') if REGEX_BOXCODE.match(input['title'])]
len(boxcodes_scraped)

In [ ]:
boxcodes_openfisca = [box for var in variables for box in var['boxes']]
len(boxcodes_openfisca)

Add some new boxes from 2023, that are not present in Openfisca yet.

In [ ]:
variables += [
    {
        'boxes': ("8HV", "8IV", "8JV", "8KV"),
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - retenue à la source sur les salaires et pensions",
    },
    {
        'boxes': ("8HW", "8IW", "8JW", "8KW"),
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - acomptes d'impôt sur le revenu",
    },
    {
        'boxes': ("8HX", "8IX", "8JX", "8KX"),
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - acomptes de prélèvements sociaux",
    },
    {
        'boxes': ("8HY", "8IY", "8JY", "8KY"),
        'type': 'int',
        'description': "Prélèvement à la source - Remboursement de trop-prélevé déjà obtenu - impôt sur le revenu",
    },
    {
        'boxes': ("8HZ", "8IZ", "8JZ", "8KZ"),
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - Remboursement de trop-prélevé déjà obtenu - prélèvements sociaux",
    },
]

The family boxes (booleans or small integer) are added manually, because:
- Their codes are ambigouous (e.g. in openfisca, there are two "F" boxes).
- The codes used in the online simulator are prefixed by the section letter: A, B, C or D (e.g. "AF" instead of "F").
- Somme boxes are not grouped together whereas they should (e.g. "AP" and "AF" are the same, but for the two partners).

In [ ]:
variables += [
    # The order is important here: the first one is the box for the first partner (wille be always used for single simulation)
    {'boxes': ('0AP', '0AF'), 'type': 'bool', 'description': "Titulaire d'une pension pour une invalidité d'au moins 40 % ou d'une carte d'invalidité d'au moins 80%"},
    {'boxes': ('0AW', '0AS'), 'type': 'bool', 'description': "Vous êtes âgé de plus de 74 ans, vous êtes titulaire de la carte du combattant ou d'une pension militaire d'invalidité ou de victime de guerre"},
    {'boxes': ('0CF',), 'type': 'int', 'description': "Nombre d'enfants à charge non mariés, qui ne sont pas en résidence alternée, de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge"},
    {'boxes': ('0CG',), 'type': 'int', 'description': "Nombre d'enfants qui ne sont pas en résidence alternée à charge titulaires de la carte d'invalidité."},
    {'boxes': ('0CH',), 'type': 'int', 'description': "Nombre d'enfants à charge en résidence alternée, non mariés de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge"},
    {'boxes': ('0CI',), 'type': 'int', 'description': "Nombre d'enfants à charge en résidence alternée titulaires de la carte d'invalidité"},
    {'boxes': ('0CR',), 'type': 'int', 'description': "Nombre de titulaires (autres que les enfants) de la carte invalidité d'au moins 80 %"},
    {'boxes': ('0DJ',), 'type': 'int', 'description': "Nombre d'enfants majeurs célibataires sans enfant"},
    {'boxes': ('0DN',), 'type': 'int', 'description': "Nombre d'enfants mariés/pacsés et d'enfants non mariés chargés de famille"},
]

The household status boxes are a particular case:

In [ ]:
variables += [
    {'boxes': ('0AM',), 'type': 'bool', 'description': "Marié·e"},
    {'boxes': ('0AO',), 'type': 'bool', 'description': "Pacsé·e"},
    {'boxes': ('0AD',), 'type': 'bool', 'description': "Divorcé·e/séparé·e"},
    {'boxes': ('0AC',), 'type': 'bool', 'description': "Célibataire"},
    {'boxes': ('0AV',), 'type': 'bool', 'description': "Veuf·ve"},
]

In [ ]:
variables += [
    {'boxes': ('5CD', '5DD', '5ED'), 'type': 'bool', 'description': "Location meublée - durée d'exercice"},
]

* Deduplicate variables (because some variables can be found in multiple places).
* Manually exclude some outdated variables (see below).

In [ ]:
EXCLUDE_VARIABLES = {
    ('5TI', '5UI', '5VI'),  # Outdated: code reused now for ('5UI', '5VI', '5WI') 
    ('3VA',),               # Old and duplicates ('3VA', '3VB', '3VO', '3VP') 
    ('3VB',),               # Old and duplicates ('3VA', '3VB', '3VO', '3VP') 
    ('3VP',),               # Outdated: code reused now for ('3VA', '3VB', '3VO', '3VP') 
}

In [ ]:
variables_dict = {}
for var in variables:
    assert isinstance(var['boxes'], tuple)
    if var['boxes'] in EXCLUDE_VARIABLES:
        print("Excluded:", var)
    else:
        variables_dict[var['boxes']] = var
variables = list(variables_dict.values())

Check duplicate boxes. If there are duplicates, they should be handled manually.

Since it means there are two different boxes with the same code (probably from different years). Use the exclusion list above.

In [ ]:
from collections import defaultdict

boxes_counts = defaultdict(int)
for var in variables:
    for box in var['boxes']:
        boxes_counts[box] += 1

duplicated_boxes = [box for box, count in boxes_counts.items() if count > 1]
print("Duplicates:", duplicated_boxes, '\n')

for var in variables:
    if any(box in duplicated_boxes for box in var['boxes']):
        print(var['boxes'], '\n', var['description'], '\n')

In [ ]:
assert not duplicated_boxes

Ajout de formulations inclusives et correction de coquilles

In [ ]:
WORD_REPLACEMENTS = [
    ('dun', "d'un"),
    ("lemploi", "l'emploi"),
    ('marié', 'marié·e'),
    ('mariés', 'marié·e·s'),
    ('âgé', 'âgé·e'),
    ('âgés', 'âgé·e·s'),
    ('pacsé', 'pacsé·e'),
    ('pacsés', 'pacsé·e·s'),
    ('né', 'né·e'),
    ('nés', 'né·e·s'),
    ('majeurs', 'majeur·e·s'),
    ('handicapé', 'handicapé·e'),
    ('handicapés', 'handicapé·e·s'),
    ('salariés', 'salarié·e·s'),
    ('pensionnés', 'pensionné·e·s'),
    ('un salarié', 'un·e salarié·e'),
    ('dun salarié', 'dun·e salarié·e'),
    ('inventeurs', 'inventeur·euse·s'),
    ('auteurs', 'auteur·ice·s'),
    ('demandeur', 'demandeur·euse'),
    ('inscrit', 'inscrit·e'),    
]
LETTER_REPLACEMENTS = [
    ('Ã©', 'é')
]
for var in variables:
    if not var['description']:
        continue
    if not var['description'].endswith('.'):
        var['description'] = var['description'] + '.'
    for lookup, repl in WORD_REPLACEMENTS:
        var['description'] = re.sub(r"\b%s\b" % lookup, repl, var['description'])
        var['description'] = re.sub(r"\b%s\b" % lookup.capitalize(), repl.capitalize(), var['description'])
    for lookup, repl in LETTER_REPLACEMENTS:
        var['description'] = var['description'].replace(lookup, repl)

In [ ]:
with open(CERFA_VARIABLES_PATH, 'w+') as f:
    json.dump(variables, f, indent=2)